<a href="https://colab.research.google.com/github/abakamousa/NER-Sangkak-challenge/blob/0.1/training/experimentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install librairies

In [1]:
!pip uninstall typing -y
!pip install flair==0.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.3/319.3 KB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 KB 50.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 KB 5.4 MB/s eta 0:00:00


In [ ]:
#!pip install flair
#!pip uninstall flair


#Import librairies

In [4]:

import pandas as pd

from flair.data import Corpus
from flair.datasets import ColumnCorpus


import flair
from typing import List
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from flair.embeddings import TransformerWordEmbeddings
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings


In [22]:
df = pd.read_csv("/content/bbj/train.txt", sep=" ")
df.tail(25)

,Msaʼnyə̂,O
47213,ntʉ́,O
47214,mnə́,O
47215,nə́,O
47216,yəŋ,O
47217,ntwɔ̂kshwɛ,O
47218,yə́,O
47219,éé,O
47220,bó,O
47221,byáp,O
47222,gaə́,O


In [5]:
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = '/content/bbj/' 
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'dev.txt')

2023-03-04 08:13:56,001 Reading data from /content/bbj
2023-03-04 08:13:56,009 Train: /content/bbj/train.txt
2023-03-04 08:13:56,015 Dev: /content/bbj/dev.txt
2023-03-04 08:13:56,017 Test: /content/bbj/test.txt


In [9]:
print("Train size: ", len(corpus.train))
print("Test size: ", len(corpus.test))
print("Dev size: ", len(corpus.dev))

Train size:  3384
Test size:  966
Dev size:  483


In [19]:
print(corpus)

Corpus: 3384 train + 483 dev + 966 test sentences


In [10]:
print(corpus.train[0].to_tagged_string('ner'))

Msaʼnyə̂ gɔtí cyətə nə́ bǎyá cyə́ nəjí pôʼ bǎhə́lə́


In [6]:
tag_type = 'ner'

#  make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

In [7]:
embedding_types = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

In [8]:
embeddings = StackedEmbeddings(embeddings=embedding_types)

# initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type=tag_type,
                        use_crf=True)

# initialize trainer
trainer = ModelTrainer(tagger, corpus)





In [9]:
embeddings_transf = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

# initialize sequence tagger
tagger_transf = SequenceTagger(hidden_size=256,
                        embeddings=embeddings_transf,
                        tag_dictionary=tag_dictionary,
                        tag_type=tag_type,
                        use_rnn=False,
                        use_crf=False)

trainer1 = ModelTrainer(tagger_transf, corpus)

In [10]:
# start training with ner-multi-fast'
trainer.train('resources/taggers/ner-multi-fast',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2023-03-04 08:14:48,303 ----------------------------------------------------------------------------------------------------
2023-03-04 08:14:48,307 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=11, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2023-03-04 08:14:48,309 ----------------------------------------------------------------------------------------------------
2023-03-04 08:14:48,315 Corpus: "Corpus: 3384 train + 483 dev + 966 test sentences"
2023-03-04 08:14:48,317 ----------------------------------------------------------------------------------------------------
2023-03-04 08:14:48,319 Parameters:
2023-03-04 08:14:48,321  - learning_

/usr/local/lib/python3.8/dist-packages/flair/trainers/trainer.py:76: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2023-03-04 08:14:50,967 epoch 1 - iter 10/106 - loss 1.79211555 - samples/sec: 123.46 - lr: 0.100000
2023-03-04 08:14:53,173 epoch 1 - iter 20/106 - loss 1.11936842 - samples/sec: 145.47 - lr: 0.100000
2023-03-04 08:14:54,886 epoch 1 - iter 30/106 - loss 0.87918186 - samples/sec: 187.24 - lr: 0.100000
2023-03-04 08:14:56,652 epoch 1 - iter 40/106 - loss 0.79523462 - samples/sec: 181.65 - lr: 0.100000
2023-03-04 08:14:58,675 epoch 1 - iter 50/106 - loss 0.78396207 - samples/sec: 158.73 - lr: 0.100000
2023-03-04 08:15:01,138 epoch 1 - iter 60/106 - loss 0.71931472 - samples/sec: 130.55 - lr: 0.100000
2023-03-04 08:15:02,895 epoch 1 - iter 70/106 - loss 0.67328444 - samples/sec: 182.93 - lr: 0.100000
2023-03-04 08:15:04,616 epoch 1 - iter 80/106 - loss 0.63288763 - samples/sec: 186.39 - lr: 0.100000
2023-03-04 08:15:06,298 epoch 1 - iter 90/106 - loss 0.60588112 - samples/sec: 190.63 - lr: 0.100000
2023-03-04 08:15:08,380 epoch 1 - iter 100/106 - loss 0.57666689 - samples/sec: 154.09 - lr

{'test_score': 0.33466666666666667,
 'dev_score_history': [0.08633093525179858,
  0.13106796116504857,
  0.16052060737527116,
  0.13842482100238662,
  0.16591928251121077,
  0.22362869198312238,
  0.19555555555555554,
  0.16444444444444445,
  0.2241014799154334,
  0.18807339449541285],
 'train_loss_history': [0.5571249644069259,
  0.3349133215764351,
  0.30188762453463974,
  0.29052446473929033,
  0.2806757074571407,
  0.2751865146722319,
  0.27033678673095934,
  0.2649503995488027,
  0.26247149711564793,
  0.26014722272027885],
 'dev_loss_history': [tensor(0.3096),
  tensor(0.2767),
  tensor(0.2542),
  tensor(0.2463),
  tensor(0.2380),
  tensor(0.2309),
  tensor(0.2298),
  tensor(0.2334),
  tensor(0.2466),
  tensor(0.2278)]}

In [ ]:
# start training with xlm-roberta-large-masakhaner

trainer1.train('resources/taggers/sota-ner-flert', #xlm-roberta-large-masakhaner',
              learning_rate=0.1,
              mini_batch_size=4,
              mini_batch_chunk_size=1,
              max_epochs=2)

2023-03-04 08:29:13,761 ----------------------------------------------------------------------------------------------------
2023-03-04 08:29:13,779 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250002, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0): XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=10